# MNIST Training using PyTorch and Step Functions

## Contents

1. [Background](#Background)
1. [Setup](#Setup)
1. [Data](#Data)
1. [Train](#Train)

---

## Background

MNIST is a widely used dataset for handwritten digit classification. It consists of 70,000 labeled 28x28 pixel grayscale images of hand-written digits. The dataset is split into 60,000 training images and 10,000 test images. There are 10 classes (one for each of the 10 digits). This tutorial will show how to train and test an MNIST model on SageMaker using PyTorch.

For more information about PyTorch in SageMaker, please visit [sagemaker-pytorch-containers](https://github.com/aws/sagemaker-pytorch-containers) and [sagemaker-python-sdk](https://github.com/aws/sagemaker-python-sdk) github repositories.

---

In [96]:
import sys
import time
import boto3
import json
!{sys.executable} -m pip install --upgrade stepfunctions

Requirement already up-to-date: stepfunctions in /home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages (1.0.0.3)
Requirement not upgraded as not directly required: boto3>=1.9.213 in /home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages (from stepfunctions) (1.10.45)
Requirement not upgraded as not directly required: sagemaker>=1.42.8 in /home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages (from stepfunctions) (1.45.0.dev0)
Requirement not upgraded as not directly required: pyyaml in /home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages (from stepfunctions) (3.12)
Requirement not upgraded as not directly required: jmespath<1.0.0,>=0.7.1 in /home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages (from boto3>=1.9.213->stepfunctions) (0.9.4)
Requirement not upgraded as not directly required: s3transfer<0.3.0,>=0.2.0 in /home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages (from boto3>=1.9.213->st

In [97]:
def createStepFunctionsWorkflowIAMRole(role_name):
    iam = boto3.client("iam")
    assume_role_policy_document = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Action": "sts:AssumeRole",
                "Principal": {"Service": "states.amazonaws.com"}
            }
        ]
    }
    
    print(json.dumps(assume_role_policy_document))
 
    role_arn = ""
    try:
        create_role_response = iam.create_role(
            RoleName = role_name,
            AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
        )
        role_arn = create_role_response["Role"]["Arn"]
    except Exception as e:
        if "EntityAlreadyExists".lower() in str(e).lower():
            print("the role already exists!")
            response = iam.list_roles(
                PathPrefix="/",
                MaxItems=1000
            )
            #print("all roles: ", response)
            for item in response["Roles"]:
                if item["RoleName"] == role_name:
                    role_arn = item["Arn"]
                    break
        else:
            print(str(e))
                    
    except:
        raise

     
        
    # AmazonPersonalizeFullAccrole_arness provides access to any S3 bucket with a name that includes "personalize" or "Personalize" 
    # if you would like to use a bucket with a different name, please consider creating and attaching a new policy
    # that provides read access to your bucket or attaching the AmazonS3ReadOnlyAccess policy to the role
    #policy_arn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
    #iam.attach_role_policy(
    #    RoleName = role_name,
    #    PolicyArn = policy_arn
    #)
    #policy_arn = "arn:aws:iam::aws:policy/AmazonS3FullAccess"
    #iam.attach_role_policy(
    #    RoleName = role_name,
    #    PolicyArn = policy_arn
    #)   
    print("pausing execution to allow for IAM role propagation.....")
    time.sleep(30) # wait to allow IAM role policy attachment to propagate

    return role_arn   

In [98]:
role_name = "AIMLBootcamp-StepFunctionsWorkflowExecutionPolicy"
role_arn = createStepFunctionsWorkflowIAMRole(role_name)
#json.dumps(create_dataset_response, indent=2)
print(role_arn)

{"Version": "2012-10-17", "Statement": [{"Effect": "Allow", "Action": "sts:AssumeRole", "Principal": {"Service": "states.amazonaws.com"}}]}
the role already exists!
pausing execution to allow for IAM role propagation.....
arn:aws:iam::247322960887:role/AIMLBootcamp-StepFunctionsWorkflowExecutionPolicy


In [99]:
PolicyDocument = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "sagemaker:CreateTransformJob",
                "sagemaker:DescribeTransformJob",
                "sagemaker:StopTransformJob",
                "sagemaker:CreateTrainingJob",
                "sagemaker:DescribeTrainingJob",
                "sagemaker:StopTrainingJob",
                "sagemaker:CreateHyperParameterTuningJob",
                "sagemaker:DescribeHyperParameterTuningJob",
                "sagemaker:StopHyperParameterTuningJob",
                "sagemaker:CreateModel",
                "sagemaker:CreateEndpointConfig",
                "sagemaker:CreateEndpoint",
                "sagemaker:DeleteEndpointConfig",
                "sagemaker:DeleteEndpoint",
                "sagemaker:UpdateEndpoint",
                "sagemaker:ListTags",
                "lambda:InvokeFunction",
                "sqs:SendMessage",
                "sns:Publish",
                "ecs:RunTask",
                "ecs:StopTask",
                "ecs:DescribeTasks",
                "dynamodb:GetItem",
                "dynamodb:PutItem",
                "dynamodb:UpdateItem",
                "dynamodb:DeleteItem",
                "batch:SubmitJob",
                "batch:DescribeJobs",
                "batch:TerminateJob",
                "glue:StartJobRun",
                "glue:GetJobRun",
                "glue:GetJobRuns",
                "glue:BatchStopJobRun"
            ],
            "Resource": "*"
        },
        {
            "Effect": "Allow",
            "Action": [
                "iam:PassRole"
            ],
            "Resource": "*",
            "Condition": {
                "StringEquals": {
                    "iam:PassedToService": "sagemaker.amazonaws.com"
                }
            }
        },
        {
            "Effect": "Allow",
            "Action": [
                "events:PutTargets",
                "events:PutRule",
                "events:DescribeRule"
            ],
            "Resource": [
                "arn:aws:events:*:*:rule/StepFunctionsGetEventsForSageMakerTrainingJobsRule",
                "arn:aws:events:*:*:rule/StepFunctionsGetEventsForSageMakerTransformJobsRule",
                "arn:aws:events:*:*:rule/StepFunctionsGetEventsForSageMakerTuningJobsRule",
                "arn:aws:events:*:*:rule/StepFunctionsGetEventsForECSTaskRule",
                "arn:aws:events:*:*:rule/StepFunctionsGetEventsForBatchJobsRule"
            ]
        }
    ]
    }

client = boto3.client("iam")

response = client.list_policies(
    Scope='Local',
    #OnlyAttached=True|False,
    #PathPrefix='string',
    #PolicyUsageFilter='PermissionsPolicy'|'PermissionsBoundary',
    #Marker='string',
    MaxItems=1000
)
#print(response)
newpolicyname = role_name + "-policy"
PolicyArn = ""
for item in response["Policies"]:
    if newpolicyname in str(item):
        print(item)
        PolicyArn = item["Arn"]
        print("PoplicyARN: ", PolicyArn)
if PolicyArn == "":
    response = client.create_policy(
        PolicyName=role_name + "-policy",
        Path="/",
        PolicyDocument=json.dumps(PolicyDocument),
        Description='AI/ML bootcamp to allow step functions to perform work'
    )
    print(response)
    PolicyArn = response["Policy"]["Arn"]

client.attach_role_policy(
    RoleName = role_name,
    PolicyArn = PolicyArn
)
  
print("pausing execution to allow for IAM role propagation.....")
time.sleep(30) # wait to allow IAM role policy attachment to propagate
print("work done: ", role_arn)

{'PolicyName': 'AIMLBootcamp-StepFunctionsWorkflowExecutionPolicy-policy', 'PolicyId': 'ANPATTFMY4P3ZJMMNLAOY', 'Arn': 'arn:aws:iam::247322960887:policy/AIMLBootcamp-StepFunctionsWorkflowExecutionPolicy-policy', 'Path': '/', 'DefaultVersionId': 'v1', 'AttachmentCount': 1, 'PermissionsBoundaryUsageCount': 0, 'IsAttachable': True, 'CreateDate': datetime.datetime(2019, 12, 31, 16, 53, 29, tzinfo=tzlocal()), 'UpdateDate': datetime.datetime(2019, 12, 31, 16, 53, 29, tzinfo=tzlocal())}
PoplicyARN:  arn:aws:iam::247322960887:policy/AIMLBootcamp-StepFunctionsWorkflowExecutionPolicy-policy
pausing execution to allow for IAM role propagation.....
work done:  arn:aws:iam::247322960887:role/AIMLBootcamp-StepFunctionsWorkflowExecutionPolicy


### Import the required modules 

Now import the required modules from the Step Functions SDK and AWS SageMaker, configure an S3 bucket, and get the AWS SageMaker execution role.

In [100]:
import sagemaker
import stepfunctions
import logging

from stepfunctions.template.pipeline import TrainingPipeline

sagemaker_session = sagemaker.Session()
stepfunctions.set_stream_logger(level=logging.INFO)

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/DEMO-pytorch-mnist'

# SageMaker Execution Role
# You can use sagemaker.get_execution_role() if running inside sagemaker's notebook instance
sagemaker_execution_role = sagemaker.get_execution_role() #Replace with ARN if not in an AWS SageMaker notebook

# paste the StepFunctionsWorkflowExecutionRole ARN from above
workflow_execution_role = role_arn 

## Data
### Getting the data



In [101]:
from torchvision import datasets, transforms

datasets.MNIST('data', download=True, transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
]))

Dataset MNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.1307,), std=(0.3081,))
           )

In [102]:
inputs = sagemaker_session.upload_data(path='data', bucket=bucket, key_prefix=prefix)
print('input spec (in this case, just an S3 path): {}'.format(inputs))

input spec (in this case, just an S3 path): s3://sagemaker-us-east-1-247322960887/sagemaker/DEMO-pytorch-mnist


### Uploading the data to S3
We are going to use the `sagemaker.Session.upload_data` function to upload our datasets to an S3 location. The return value inputs identifies the location -- we will use later when we start the training job.


## Train
### Training script
The `mnist.py` script provides all the code we need for training and hosting a SageMaker model (`model_fn` function to load a model).
The training script is very similar to a training script you might run outside of SageMaker, but you can access useful properties about the training environment through various environment variables, such as:

* `SM_MODEL_DIR`: A string representing the path to the directory to write model artifacts to.
  These artifacts are uploaded to S3 for model hosting.
* `SM_NUM_GPUS`: The number of gpus available in the current container.
* `SM_CURRENT_HOST`: The name of the current container on the container network.
* `SM_HOSTS`: JSON encoded list containing all the hosts .

Supposing one input channel, 'training', was used in the call to the PyTorch estimator's `fit()` method, the following will be set, following the format `SM_CHANNEL_[channel_name]`:

* `SM_CHANNEL_TRAINING`: A string representing the path to the directory containing data in the 'training' channel.

For more information about training environment variables, please visit [SageMaker Containers](https://github.com/aws/sagemaker-containers).

A typical training script loads data from the input channels, configures training with hyperparameters, trains a model, and saves a model to `model_dir` so that it can be hosted later. Hyperparameters are passed to your script as arguments and can be retrieved with an `argparse.ArgumentParser` instance.

Because the SageMaker imports the training script, you should put your training code in a main guard (``if __name__=='__main__':``) if you are using the same script to host your model as we do in this example, so that SageMaker does not inadvertently run your training code at the wrong point in execution.

For example, the script run by this notebook:

In [103]:
!pygmentize mnist.py

import argparse
import json
import logging
import os
import sagemaker_containers
import sys
import torch
import torch.distributed as dist
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
import torch.utils.data.distributed
from torchvision import datasets, transforms

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler(sys.stdout))


# Based on https://github.com/pytorch/examples/blob/master/mnist/main.py
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)

### Use Step Functions to run training in SageMaker

The `PyTorch` class allows us to run our training function as a training job on SageMaker. We need to configure it with our training script, an IAM role, the number of training instances, the training instance type, and hyperparameters. In this case we are going to run our training job on 2 ```ml.c4.xlarge``` instances. But this example can be ran on one or multiple, cpu or gpu instances ([full list of available instances](https://aws.amazon.com/sagemaker/pricing/instance-types/)). The hyperparameters parameter is a dict of values that will be passed to your training script -- you can see how to access these values in the `mnist.py` script above.


In [104]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point='mnist.py',
                    role=sagemaker_execution_role,
                    framework_version='1.2.0',
                    train_instance_count=2,
                    train_instance_type='ml.c4.xlarge',
                    hyperparameters={
                        'epochs': 6,
                        'backend': 'gloo'
                    })

### Build a training pipeline with the Step Functions SDK

A typical task for a data scientist is to train a model and deploy that model to an endpoint. Without the Step Functions SDK, this is a four step process on SageMaker that includes the following.

1. Training the model
2. Creating the model on SageMaker
3. Creating an endpoint configuration
4. Deploying the trained model to the configured endpoint

The Step Functions SDK provides the [TrainingPipeline](https://aws-step-functions-data-science-sdk.readthedocs.io/en/latest/pipelines.html#stepfunctions.template.pipeline.train.TrainingPipeline) API to simplify this procedure. The following configures `pipeline` with the necessary parameters to define a training pipeline.


In [105]:
pipeline = TrainingPipeline(
    estimator=estimator,
    role=workflow_execution_role,
    inputs=inputs,
    s3_bucket=bucket
)

### Visualize the pipeline

You can now view the workflow definition, and also visualize it as a graph. This workflow and graph represent your training pipeline.

#### View the workflow definition

In [106]:
print(pipeline.workflow.definition.to_json(pretty=True))

{
    "StartAt": "Training",
    "States": {
        "Training": {
            "Resource": "arn:aws:states:::sagemaker:createTrainingJob.sync",
            "Parameters": {
                "AlgorithmSpecification.$": "$$.Execution.Input['Training'].AlgorithmSpecification",
                "OutputDataConfig.$": "$$.Execution.Input['Training'].OutputDataConfig",
                "StoppingCondition.$": "$$.Execution.Input['Training'].StoppingCondition",
                "ResourceConfig.$": "$$.Execution.Input['Training'].ResourceConfig",
                "RoleArn.$": "$$.Execution.Input['Training'].RoleArn",
                "InputDataConfig.$": "$$.Execution.Input['Training'].InputDataConfig",
                "HyperParameters.$": "$$.Execution.Input['Training'].HyperParameters",
                "TrainingJobName.$": "$$.Execution.Input['Training'].TrainingJobName"
            },
            "Type": "Task",
            "Next": "Create Model"
        },
        "Create Model": {
            "Par

#### Visualize the workflow graph

In [107]:
pipeline.render_graph()

### Create and execute the pipeline on AWS Step Functions

Create the pipeline in AWS Step Functions with [create](https://aws-step-functions-data-science-sdk.readthedocs.io/en/latest/workflow.html#stepfunctions.workflow.Workflow.create).

In [108]:
pipeline.create()

[INFO] Workflow created successfully on AWS Step Functions.


'arn:aws:states:us-east-1:247322960887:stateMachine:training-pipeline-2019-12-31-16-55-54'

Run the workflow with [execute](https://aws-step-functions-data-science-sdk.readthedocs.io/en/latest/workflow.html#stepfunctions.workflow.Workflow.execute). A link will be provided after the following cell is executed. Following this link, you can monitor your pipeline execution on Step Functions' console.

In [109]:
pipeline.execute()

[INFO] Workflow execution started successfully on AWS Step Functions.


Execution(execution_arn='arn:aws:states:us-east-1:247322960887:execution:training-pipeline-2019-12-31-16-55-54:training-pipeline-2019-12-31-16-55-54', name=None, status=<ExecutionStatus.Running: 'RUNNING'>, start_date=datetime.datetime(2019, 12, 31, 16, 55, 54, 749000, tzinfo=tzlocal()))